# NLP and RNN
In this notebook we will check our understanding of the concepts learned in NLP and RNN.

![](https://drive.google.com/uc?export=view&id=1ZtBJcNXO-BQhK86joluZYzN7b1GHYLLs)

To summarize NLP or Natural Language Processing is:

* Computer manipulation of natural languages.
* Set of methods/algorithms for making natural language accessible to computer.


The image below summarizes the basic steps involved in any NLP task:

![](https://drive.google.com/uc?export=view&id=1PNXHplPzlWnrddeh5oPX6ZuJSfgnuHCa)



There are 5 exercises in total and an optional exercise. To answer some of the exercises (3, 4 and 5) you will be required to write a code from scratch in the code cells containing:

`# write your code here`


Before starting make sure you are using the GPU.

In [ ]:
!nvidia-smi

Sun Mar 21 17:34:15 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Tokenization


We will use TensorFlow Keras Tokenizer to tokenize our text. As per the [TensorFlow documentation](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer):

“This class allows to vectorize a text corpus, by turning each text into either a sequence of integers (each integer being the index of a token in a dictionary) or into a vector where the coefficient for each token could be binary, based on word count, based on tf-idf.”


There are many functions that we can use, but below we will be using these two functions to train the tokenizer to our text data and convert given text to tokens:

* `fit_on_texts`: Updates internal vocabulary based on a list of texts. This method creates the vocabulary index based on word frequency. So if you give it something like, "The cat sat on the mat." It will create a dictionary “word_index” such that every word gets a unique integer value. 0 is reserved for padding. So lower integer means more frequent word.

* `texts_to_sequences` Transforms each text in texts to a sequence of integers. It takes each word in the text and replaces it with its corresponding integer value from the word_index dictionary. 




In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
t  = Tokenizer()
fit_text = ["In science, you can say things that seem crazy, but in the long run, they can turn out to be right"]
t.fit_on_texts(fit_text)


test_text1 = "I would like to take a right turn"
test_text2 = "That man is crazy"
sequences = t.texts_to_sequences([test_text1, test_text2])

print('sequences : ',sequences,'\n')

print('word_index : ',t.word_index)

sequences :  [[17, 19, 15], [7, 9]] 

word_index :  {'in': 1, 'can': 2, 'science': 3, 'you': 4, 'say': 5, 'things': 6, 'that': 7, 'seem': 8, 'crazy': 9, 'but': 10, 'the': 11, 'long': 12, 'run': 13, 'they': 14, 'turn': 15, 'out': 16, 'to': 17, 'be': 18, 'right': 19}


#### Exercise 1
In the code above we tokenize two sentences:
* "I would like to take a right turn"
* "That man is crazy"

a. What is the tokenized version of these sentences?


b. The first sentence has 8 words, and second sentence has 4 words, however the tokenized version has 3 and 2 integers respectively for them. Why is it so?

**Answer 1**: 

A. ["I","would","like","to","take","a","right","turn"] and ["That","man","is","crazy"]

## Embeddings


#### Exercise 2
In the class we learned about embeddings, let us explore them a little more. Kindly go to the site [Embeddings Projector](http://projector.tensorflow.org). Play around a bit and answer the following questions:

1.  For the word 'fantastic' list the five nearest neighbours, when using `Word2Vec 10K` embedding.
2. Repeat the exercise by changing the embeddings to `Word2Vec All`.

Reflect on the result. How do you think the world `fantastic` is related to its five nearest neighbours?

**Answer 2**: 

1. In descending order: marvel, spider, amazing, strange, weird, comics, daredevil, storyline

2. Fantastic, Fantastical, Fantastically


Although they do not have a direct relationship in terms of meaning, these words can be related according to the objective of the sentence. That is, they could, a sentence could have hinted that daredevil is fantastic, for example. In question 2. The results are directly related to the word "fantastic".


## Word Similarities
Let us now train Word2Vec model on text8 dataset

In [2]:
!mkdir data

import gensim.downloader as api
from gensim.models import Word2Vec

info = api.info("text8")
assert(len(info) > 0)

dataset = api.load("text8")  # download and load text 8  dataset
model = Word2Vec(dataset) # we create an embedding using Word2vec model for this data

model.save("data/text8-word2vec.bin")

[==================================================] 100.0% 31.6/31.6MB downloaded


#### Load the saved model as KeyedVector to save space.

In [3]:
from gensim.models import KeyedVectors
model = KeyedVectors.load("data/text8-word2vec.bin")  # Help in saving memory by shedding the internal data structures necessary for training
word_vectors = model.wv   ## Gives the word vectors

#### Helper function to print

In [4]:
def print_most_similar(word_conf_pairs, k):
    for i, (word, conf) in enumerate(word_conf_pairs):
        print("{:.3f} {:s}".format(conf, word))
        if i >= k-1:
            break
    if k < len(word_conf_pairs):
        print("...")

In [5]:
print_most_similar(word_vectors.most_similar('king'),5)

0.725 prince
0.719 emperor
0.695 queen
0.694 throne
0.687 vii
...


#### Exercise 3
In the class, we learned how to use the Word2Vec embeddings in Gensim. When the model is trained on the ‘text8’ dataset, give five most similar words to the word ‘tree’ using word2vec embedding trained on ‘text8’ dataset.

In [6]:
## Write your code here

print_most_similar(word_vectors.most_similar('tree'),5)

0.701 trees
0.686 leaf
0.661 bark
0.649 vine
0.644 fruit
...


**Answer 3**:

trees, bark, leaf, avl, fruit

## Word Arithmetics




#### Exercise 4
With the Word2Vec model trained on text8 dataset, calculate the following:

*	woman + king - man = ?
*	chair + table - work = ?
*	Queens - queen + person = ?

In [7]:
## Write your code here
result = model.most_similar(positive=['woman', 'king'], negative=['man'], topn=1);
print(result);

result = model.most_similar(positive=['chair', 'table'], negative=['work'], topn=1);
print(result);

result = model.most_similar(positive=['queens', 'person'], negative=['queen'], topn=1);
print(result);

[('queen', 0.6665854454040527)]
[('bar', 0.6440656185150146)]
[('node', 0.5654397010803223)]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


**Answer 4**: (Double Click to edit)

*	woman + king - man = queen
*	chair + table - work = bar
*	Queens - queen + person = finite

## Spam Classifier

Some helper codes:
* importing required modules
* defining helper functions
* Building model

The code cells below are  hidden, that is by default you cannot see the code in them, but remember to run these cells. You can check the code by double clicking the cells.

In [8]:
#@title
# The modules needed to run the code
import argparse  # To read commandline argument and parse it
import gensim.downloader as api
import numpy as np
import os  # For file and directory handling
import shutil  # For file and directory handling
import tensorflow as tf

from sklearn.metrics import accuracy_score, confusion_matrix  #For measuring performance

# Some parameters
DATA_DIR = "data"   # Data directory to save embedding
EMBEDDING_NUMPY_FILE = os.path.join(DATA_DIR, "E.npy")  # Numpy file containing word embeddings
DATASET_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip"  # Dataset URL from where data is downloaded
EMBEDDING_MODEL = "glove-wiki-gigaword-300"  # The gensim embedding model we will use
EMBEDDING_DIM = 300  # The embedding dimensions
NUM_CLASSES = 2  # The number of classes in output-- Spam or Ham
BATCH_SIZE = 128  # The batch size
NUM_EPOCHS = 3  # number of epochs for which model is to be trained


# data distribution is 4827 ham and 747 spam (total 5574), which 
# works out to approx 87% ham and 13% spam, so we take reciprocals
# and this works out to being each spam (1) item as being approximately
# 8 times as important as each ham (0) message.
CLASS_WEIGHTS = { 0: 1, 1: 8 }  # To take care of imbalance in classes

tf.random.set_seed(42)  # Set the seed for random number generation to be able to reproduce results. 


# Data downloading and data Processing



def download_and_read(url):
    """
    The function downloads the data from given url, splits it into Text and Labels
    Uses tf.keras.utils.get_file() function to download the data from url--> function 
    downloads the data from the given url, extracts it from the zip file and place it in folder "datasets" 
    with the name specified in the first argument.
    tf.keras.utils.get_file(
    fname, origin, untar=False, md5_hash=None, file_hash=None,
    cache_subdir='datasets', hash_algorithm='auto', extract=False,
    archive_format='auto', cache_dir=None)

    Arguments:
    url: The url link of the dataset in zip format

    Returns:
    Two lists containing texts and respective labels

    """
    local_file = url.split('/')[-1]  # split the file name (last string after '/') from url
    p = tf.keras.utils.get_file(local_file, url, 
        extract=True, cache_dir=".")  #function to download the data from url to folder datasets with name given in local_file
    labels, texts = [], []
    local_file = os.path.join("datasets", "SMSSpamCollection")  # define the path of the file from which to read data: datasets/SMSSpamCollection
    with open(local_file, "r") as fin:
        for line in fin:
            label, text = line.strip().split('\t')  # The labels and text are in one line separated by tab space.
            labels.append(1 if label == "spam" else 0)
            texts.append(text)
    return texts, labels

def build_embedding_matrix(sequences, word2idx, embedding_dim, 
        embedding_file):
    """
    The function reads the dict word2idx (word --> number) and written the corresponding
    word vector for each word as defined by the Embedding model

    Arguments:
    sequences: not needed, not used-- just there because to suport back support for TF1 book
    word2idx: Dictionary  containing words in the text and their respective idx as given by tokenizer.
    embedding_dim: The number of units for the embedding layer
    embedding_file: The data file in which embeddings will be store for future use.

    """
    if os.path.exists(embedding_file):  # Checks if the embedding file already exists- then it justs loads it in the memory
        E = np.load(embedding_file)
    else:  # Else it creates the embedding file using the model specified in EMBEDDING_MODEL
        vocab_size = len(word2idx)  # The vocabulary size is number of unique words in the text
        E = np.zeros((vocab_size, embedding_dim)) # Creates a variable to store embeddings
        word_vectors = api.load(EMBEDDING_MODEL)  # Get the embeddings from Gensim
        for word, idx in word2idx.items():
            try:
                E[idx] = word_vectors.word_vec(word)  # For each word it converts it to respective word vector and store in Embedding file
            except KeyError:   # word not in embedding
                pass
            # except IndexError: # UNKs are mapped to seq over VOCAB_SIZE as well as 1
            #     pass
        np.save(embedding_file, E)  # The embeddings are saved in a file for future reference
    return E

In [9]:
#@title
class SpamClassifierModel(tf.keras.Model):  # The model is build using model API of Keras with tf.Keras.Model as the parent class. 
# The class inherits train, predict methods of the parent class.
    def __init__(self, vocab_sz, embed_sz, input_length,
            num_filters, kernel_sz, output_sz, 
            run_mode, embedding_weights, 
            **kwargs):
        super(SpamClassifierModel, self).__init__(**kwargs)
        if run_mode == "scratch":  # Choose the embedding layer scratch means the weights wil be traned from scratch
            self.embedding = tf.keras.layers.Embedding(vocab_sz, 
                embed_sz,
                input_length=input_length,
                trainable=True)
        elif run_mode == "vectorizer":  # Vectorizer means we use the pre-trained weights--> Transfer Learning
            self.embedding = tf.keras.layers.Embedding(vocab_sz, 
                embed_sz,
                input_length=input_length,
                weights=[embedding_weights],
                trainable=False)
        else:  # This is the fine tuning mode- we use pre-trained weights for the embedding layer and fine tune them. 
            self.embedding = tf.keras.layers.Embedding(vocab_sz, 
                embed_sz,
                input_length=input_length,
                weights=[embedding_weights],
                trainable=True)
        self.dropout = tf.keras.layers.SpatialDropout1D(0.2)  # Add droput layer to avoid overfotting. 
        self.conv = tf.keras.layers.Conv1D(filters=num_filters,  # Define the 1D convolutional layer 
            kernel_size=kernel_sz,
            activation="relu")
        self.pool = tf.keras.layers.GlobalMaxPooling1D()  # The pooling layer
        self.dense = tf.keras.layers.Dense(output_sz, 
            activation="softmax")  # And the last classifying layer consists of a fully connected Dense layer

    def call(self, x):  # This function performs forward pass in the model. 
        x = self.embedding(x)
        x = self.dropout(x)
        x = self.conv(x)
        x = self.pool(x)
        x = self.dense(x)
        return x

In [10]:
#@title
# The code below requires a folder to be created
!mkdir data

## Now we will use the functions and model defined above --> ideally they should be done in a separate file-- main.py

# read data
texts, labels = download_and_read(DATASET_URL)

# tokenize and pad text so that each text is of same size
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(texts)
text_sequences = tokenizer.texts_to_sequences(texts)
text_sequences = tf.keras.preprocessing.sequence.pad_sequences(text_sequences)
num_records = len(text_sequences)
max_seqlen = len(text_sequences[0])
#print("{:d} sentences, max length: {:d}".format(num_records, max_seqlen))

# labels --> convert labels to categorical labels (one hot encoded)
cat_labels = tf.keras.utils.to_categorical(labels, num_classes=NUM_CLASSES)

# vocabulary --> Create word mapping and its inverse
word2idx = tokenizer.word_index
idx2word = {v:k for k, v in word2idx.items()}
word2idx["PAD"] = 0
idx2word[0] = "PAD"
vocab_size = len(word2idx)
#print("vocab size: {:d}".format(vocab_size))

# load the dataset as tensors, split it into test, train and validation set
dataset = tf.data.Dataset.from_tensor_slices((text_sequences, cat_labels))
dataset = dataset.shuffle(10000)
test_size = num_records // 4
val_size = (num_records - test_size) // 10
test_dataset = dataset.take(test_size)
val_dataset = dataset.skip(test_size).take(val_size)
train_dataset = dataset.skip(test_size + val_size)

test_dataset = test_dataset.batch(BATCH_SIZE, drop_remainder=True)
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=True)

# Build the embedding
E = build_embedding_matrix(text_sequences, word2idx, EMBEDDING_DIM,
    EMBEDDING_NUMPY_FILE)
#print("Embedding matrix:", E.shape)

#Since we are not passing the mode by command line in this file we need to give a value to run_mode
run_mode = 'scratch'


mkdir: cannot create directory ‘data’: File exists
204800/203415 [==============================] - 0s 1us/step
[==================================================] 100.0% 376.1/376.1MB downloaded


In [11]:
# Now we use the SpamClassifierModel class to create a model
conv_num_filters = 256
conv_kernel_size = 3
model = SpamClassifierModel(
    vocab_size, EMBEDDING_DIM, max_seqlen, 
    conv_num_filters, conv_kernel_size, NUM_CLASSES,
    run_mode, E)
model.build(input_shape=(None, max_seqlen))
model.summary()

Model: "spam_classifier_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  2703000   
_________________________________________________________________
spatial_dropout1d (SpatialDr multiple                  0         
_________________________________________________________________
conv1d (Conv1D)              multiple                  230656    
_________________________________________________________________
global_max_pooling1d (Global multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  514       
Total params: 2,934,170
Trainable params: 2,934,170
Non-trainable params: 0
_________________________________________________________________


#### Compile and train the model

In [12]:
# Define  compile and train
model.compile(optimizer="adam", loss="categorical_crossentropy",
    metrics=["accuracy"])

# Now we train the model
model.fit(train_dataset, epochs=NUM_EPOCHS, 
    validation_data=val_dataset,
    class_weight=CLASS_WEIGHTS)

Epoch 1/3
29/29 [==============================] - 34s 64ms/step - loss: 1.1398 - accuracy: 0.4778 - val_loss: 0.2144 - val_accuracy: 0.9896
Epoch 2/3
29/29 [==============================] - 1s 48ms/step - loss: 0.3875 - accuracy: 0.9693 - val_loss: 0.0552 - val_accuracy: 0.9792
Epoch 3/3
29/29 [==============================] - 1s 48ms/step - loss: 0.0850 - accuracy: 0.9913 - val_loss: 0.0539 - val_accuracy: 0.9818


### And now we evaluate the model on test dataset.

In [13]:
# Lastly we evaluate the trained model against test set
labels, predictions = [], []
for Xtest, Ytest in test_dataset:  
    Ytest_ = model.predict_on_batch(Xtest)  # for each test test predict the label
    ytest = np.argmax(Ytest, axis=1)  # Get the label with highest probabilty from actual test output
    ytest_ = np.argmax(Ytest_, axis=1) # Get the label with highest probabilty from predictted test output
    labels.extend(ytest.tolist())  # add to list
    predictions.extend(ytest.tolist())  # add to list

print("test accuracy: {:.3f}".format(accuracy_score(labels, predictions)))  # Calculate accuracy score

test accuracy: 1.000


#### Exercise 5
In the spam classifier what is the false positive and false negative on the test dataset?  What does it tell you about the trained model?

In [14]:
# Write your code here
import numpy as np
from sklearn.metrics import confusion_matrix

print(np.unique(labels))
print(np.unique(predictions))

print(confusion_matrix(labels, predictions))



[0 1]
[0 1]
[[1091    0]
 [   0  189]]


**Answer 5**: 
False positive and false negative are zero (0). According to the test accuracy, the model classifies all the instances correctly.  While this may be positive, this could also be a sign of overfitting.

# Optional Exercise
Consider the Tweet sentiment dataset from Stanford: http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip Which fields you will require for training? Modify the `download_and_read(url)` function to read from this URL and return the necessary labeled sentences.

What is the accuracy of your twitter sentiment model after 5 epochs, after 10 epochs, after 15 epochs? Do you think it should be trained for more than 15 epochs? Give reasoning for your answer. 

In the class we used bi-LSTM, try change the recurrent network from bi-LSTM to GRU, how does it change the model performance?

In [ ]:
#@title
# The modules needed to run the code
import argparse  # To read commandline argument and parse it
import gensim.downloader as api
import numpy as np
import os  # For file and directory handling
import shutil  # For file and directory handling
import tensorflow as tf

from sklearn.metrics import accuracy_score, confusion_matrix  #For measuring performance

# Some parameters
DATA_DIR = "data_2"   # Data directory to save embedding
EMBEDDING_NUMPY_FILE = os.path.join(DATA_DIR, "E.npy")  # Numpy file containing word embeddings
DATASET_URL = "http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip"  # Dataset URL from where data is downloaded
EMBEDDING_MODEL = "glove-wiki-gigaword-300"  # The gensim embedding model we will use
EMBEDDING_DIM = 300  # The embedding dimensions
NUM_CLASSES = 2  # The number of classes in output-- Spam or Ham
BATCH_SIZE = 128  # The batch size
NUM_EPOCHS = 3  # number of epochs for which model is to be trained


# data distribution is 4827 ham and 747 spam (total 5574), which 
# works out to approx 87% ham and 13% spam, so we take reciprocals
# and this works out to being each spam (1) item as being approximately
# 8 times as important as each ham (0) message.
CLASS_WEIGHTS = { 0: 1, 1: 8 }  # To take care of imbalance in classes

tf.random.set_seed(42)  # Set the seed for random number generation to be able to reproduce results. 



In [ ]:
#@title
# The code below requires a folder to be created
!mkdir data_2

## Now we will use the functions and model defined above --> ideally they should be done in a separate file-- main.py

# read data
texts, labels = download_and_read(DATASET_URL)

# tokenize and pad text so that each text is of same size
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(texts)
text_sequences = tokenizer.texts_to_sequences(texts)
text_sequences = tf.keras.preprocessing.sequence.pad_sequences(text_sequences)
num_records = len(text_sequences)
max_seqlen = len(text_sequences[0])
#print("{:d} sentences, max length: {:d}".format(num_records, max_seqlen))

# labels --> convert labels to categorical labels (one hot encoded)
cat_labels = tf.keras.utils.to_categorical(labels, num_classes=NUM_CLASSES)

# vocabulary --> Create word mapping and its inverse
word2idx = tokenizer.word_index
idx2word = {v:k for k, v in word2idx.items()}
word2idx["PAD"] = 0
idx2word[0] = "PAD"
vocab_size = len(word2idx)
#print("vocab size: {:d}".format(vocab_size))

# load the dataset as tensors, split it into test, train and validation set
dataset = tf.data.Dataset.from_tensor_slices((text_sequences, cat_labels))
dataset = dataset.shuffle(10000)
test_size = num_records // 4
val_size = (num_records - test_size) // 10
test_dataset = dataset.take(test_size)
val_dataset = dataset.skip(test_size).take(val_size)
train_dataset = dataset.skip(test_size + val_size)

test_dataset = test_dataset.batch(BATCH_SIZE, drop_remainder=True)
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=True)

# Build the embedding
E = build_embedding_matrix(text_sequences, word2idx, EMBEDDING_DIM,
    EMBEDDING_NUMPY_FILE)
#print("Embedding matrix:", E.shape)

#Since we are not passing the mode by command line in this file we need to give a value to run_mode
run_mode = 'scratch'

In [ ]:
# using the same model
conv_num_filters = 256
conv_kernel_size = 3
model = SpamClassifierModel(
    vocab_size, EMBEDDING_DIM, max_seqlen, 
    conv_num_filters, conv_kernel_size, NUM_CLASSES,
    run_mode, E)
model.build(input_shape=(None, max_seqlen))
model.summary()

Model: "spam_classifier_model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      multiple                  2703000   
_________________________________________________________________
spatial_dropout1d_3 (Spatial multiple                  0         
_________________________________________________________________
conv1d_3 (Conv1D)            multiple                  230656    
_________________________________________________________________
global_max_pooling1d_3 (Glob multiple                  0         
_________________________________________________________________
dense_3 (Dense)              multiple                  514       
Total params: 2,934,170
Trainable params: 2,934,170
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Define  compile and train
model.compile(optimizer="adam", loss="categorical_crossentropy",
    metrics=["accuracy"])

NUM_EPOCHS = [5,10,15]

In [ ]:
# Now we train the model

for i in range(0,len(NUM_EPOCHS)):
  model.fit(train_dataset, epochs=NUM_EPOCHS[i], 
  validation_data=val_dataset,
  class_weight=CLASS_WEIGHTS)
  
  labels, predictions = [], []
  for Xtest, Ytest in test_dataset:  
    Ytest_ = model.predict_on_batch(Xtest)  # for each test test predict the label
    ytest = np.argmax(Ytest, axis=1)  # Get the label with highest probabilty from actual test output
    ytest_ = np.argmax(Ytest_, axis=1) # Get the label with highest probabilty from predictted test output
    labels.extend(ytest.tolist())  # add to list
    predictions.extend(ytest.tolist())  # add to list
  print("test accuracy: {:.3f}".format(accuracy_score(labels, predictions)))  # Calculate accuracy score

Epoch 1/5
29/29 [==============================] - 2s 61ms/step - loss: 1.1398 - accuracy: 0.4778 - val_loss: 0.2144 - val_accuracy: 0.9896
Epoch 2/5
29/29 [==============================] - 1s 50ms/step - loss: 0.3875 - accuracy: 0.9693 - val_loss: 0.0552 - val_accuracy: 0.9792
Epoch 3/5
29/29 [==============================] - 1s 48ms/step - loss: 0.0850 - accuracy: 0.9913 - val_loss: 0.0539 - val_accuracy: 0.9818
Epoch 4/5
29/29 [==============================] - 1s 48ms/step - loss: 0.0631 - accuracy: 0.9911 - val_loss: 0.0142 - val_accuracy: 0.9974
Epoch 5/5
29/29 [==============================] - 1s 46ms/step - loss: 0.0323 - accuracy: 0.9980 - val_loss: 0.0119 - val_accuracy: 1.0000
test accuracy: 1.000
test accuracy: 1.000
test accuracy: 1.000
test accuracy: 1.000
test accuracy: 1.000
test accuracy: 1.000
test accuracy: 1.000
test accuracy: 1.000
test accuracy: 1.000
test accuracy: 1.000
Epoch 1/10
29/29 [==============================] - 1s 48ms/step - loss: 0.0143 - accuracy